# Modelling

In [98]:
import pandas as pd
import numpy as np

from scipy import sparse

from lightfm import LightFM
from sklearn.metrics.pairwise import cosine_similarity
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

from scipy.spatial import distance

from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors


from resources import *

#Import Warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
%run resources.py

## Load data

In [3]:
# Load user items data
recdata = pd.read_csv('../Data/steam/recdata.csv', index_col=0)
recdata = recdata.rename(columns = {'variable':'id', 'value': 'owned'})
recdata.head()

,id,uid,owned
0,10,0,1.0
1,10,1,1.0
2,10,3,1.0
3,10,4,1.0
4,10,10,1.0


In [4]:
# Load game names data
games = pd.read_csv('../Data/steam/gamenames.csv', index_col = 0)
games.head()

,title,id
0,Lost Summoner Kitty,761140.0
1,Ironbound,643980.0
2,Real Pool 3D - Poolians,670290.0
3,弹炸人2222,767400.0
4,NaN,773570.0


In [5]:
# Load game names data
gamesdata = pd.read_csv('gamesdata.csv', index_col = 0)
gamesdata.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.49,http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140.0,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980.0,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,False,670290.0,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",0.83,http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,False,767400.0,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"['Action', 'Indie', 'Casual', 'Sports']",1.79,http://steamcommunity.com/app/773570/reviews/?...,"['Single-player', 'Full controller support', '...",2.99,False,773570.0,NaN,NaN,NaN


## Additional Preprocessing

### Create interaction matrix

In [6]:
# Use create_interaction_matrix function
interactions = create_interaction_matrix(df = recdata,
                                         user_col = 'uid',
                                         item_col = 'id',
                                         rating_col = 'owned')
interactions.shape

(69277, 8791)

In [7]:
interactions.head()

id,10,20,30,40,50,60,70,80,130,220,...,526790,527340,527440,527510,527520,527810,527890,527900,528660,530720
uid,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
type(interactions)

pandas.core.frame.DataFrame

### Train test split

In [9]:
# Get number of users
len(interactions)

69277

In [10]:
# Establish number of users in train/test sets

train_num = round((80/100)*len(interactions),0)
print(f'We desire {train_num} users in our training set.')

test_num = len(interactions)-train_num
print(f'We desire {test_num} users in our test set.')

We desire 55422.0 users in our training set.
We desire 13855.0 users in our test set.


In [11]:
# Define train and test sets
train = interactions[:55422]
test = interactions[55422:]

### Create user dictionary

In [12]:
# Create user dictionary using helper function
user_dict = create_user_dict(interactions=interactions)

### Create item dictionary

In [13]:
# Create game dictionary using helper function
games_dict = create_item_dict(df = gamesdata, id_col = 'id', name_col = 'title')

### Create sparse matrices

In [14]:
# Create sparse matrices for evaluation 
train_sparse = sparse.csr_matrix(train.values)

#Add X users to Test so that the number of rows in Train match Test
N = train.shape[0] #Rows in Train set
n,m = test.shape #Rows & columns in Test set
z = np.zeros([(N-n),m]) #Create the necessary rows of zeros with m columns
#test = test.toarray() #Temporarily convert Test into a numpy array
test = np.vstack((test,z)) #Vertically stack Test on top of the blank users
test_sparse = sparse.csr_matrix(test) #Convert back to sparse

## Modelling using LightFM

### WARP loss model

In [15]:
# Instantiate and fit model
mf_model_warp = run_model(interactions = train,
                 n_components = 30,
                 loss = 'warp',
                 epoch = 30,
                 n_jobs = 4)

In [16]:
# Get precision
train_precision = precision_at_k(mf_model_warp, train_sparse, k=10).mean()
test_precision = precision_at_k(mf_model_warp, test_sparse, k=10).mean()
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.66, test 0.19.


In [17]:
# Get AUC
train_auc = auc_score(mf_model_warp, train_sparse).mean()
test_auc = auc_score(mf_model_warp, test_sparse).mean()
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

AUC: train 0.99, test 0.94.


### BPR loss model

In [18]:
# Instantiate and fit model
mf_model_bpr = run_model(interactions = train,
                 n_components = 30,
                 loss = 'bpr',
                 epoch = 30,
                 n_jobs = 4)

In [19]:
# Get precision
train_precision = precision_at_k(mf_model_bpr, train_sparse, k=10).mean()
test_precision = precision_at_k(mf_model_bpr, test_sparse, k=10).mean()
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.75, test 0.15.


In [20]:
# Get AUC
train_auc = auc_score(mf_model_bpr, train_sparse).mean()
test_auc = auc_score(mf_model_bpr, test_sparse).mean()
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

AUC: train 0.97, test 0.73.


### Adjust num of components

In [21]:
# Instantiate and fit model
mf_model_warp_2 = run_model(interactions = train,
                 n_components = 10,
                 loss = 'warp',
                 epoch = 30,
                 n_jobs = 4)

In [22]:
# Get precision
train_precision = precision_at_k(mf_model_warp_2, train_sparse, k=10).mean()
test_precision = precision_at_k(mf_model_warp_2, test_sparse, k=10).mean()
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.60, test 0.22.


In [23]:
# Get AUC
train_auc = auc_score(mf_model_warp_2, train_sparse).mean()
test_auc = auc_score(mf_model_warp_2, test_sparse).mean()
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

AUC: train 0.98, test 0.95.


In [24]:
# Instantiate and fit model
mf_model_warp_50 = run_model(interactions = train,
                 n_components = 50,
                 loss = 'warp',
                 epoch = 30,
                 n_jobs = 4)

In [25]:
# Get precision
train_precision = precision_at_k(mf_model_warp_50, train_sparse, k=10).mean()
test_precision = precision_at_k(mf_model_warp_50, test_sparse, k=10).mean()
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.70, test 0.18.


In [26]:
# Get AUC
train_auc = auc_score(mf_model_warp_50, train_sparse).mean()
test_auc = auc_score(mf_model_warp_50, test_sparse).mean()
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

AUC: train 0.99, test 0.94.


### Final model

In [27]:
# Instantiate and fit model on full interactions set
mf_model = run_model(interactions = interactions,
                 n_components = 30,
                 loss = 'warp',
                 epoch = 30,
                 n_jobs = 4)

## Embeddings

### Retrieve embeddings matrix

In [31]:
# Get embeddings
embeddings = mf_model.item_embeddings
embeddings

array([[ 0.24690884, -0.60024494, -0.34321675, ...,  0.39954203,
        -0.34470356,  0.6996237 ],
       [ 1.013957  , -1.0958432 , -1.1673098 , ...,  1.5129188 ,
        -1.0025263 ,  0.8605353 ],
       [ 1.419911  , -0.76011163, -1.4812998 , ...,  1.0426521 ,
        -0.58840156,  0.6917034 ],
       ...,
       [ 0.37666148,  0.06383109, -0.37983808, ...,  0.01275835,
        -0.647543  , -0.5578531 ],
       [ 0.31045482,  0.4377617 , -0.30335838, ..., -0.27258068,
        -0.25924444, -0.29798537],
       [ 0.8851168 , -0.1208554 , -0.43298256, ..., -0.08469868,
        -0.27769536, -0.29746038]], dtype=float32)

In [32]:
# View shape
embeddings.shape

(8791, 30)

Our embeddings array has 8791 rows, with each row representing a game. Each row has 30 numbers, representing the components of our game embeddings. This is because we set `n_components` to be 30 in our model.

Let's investigate a sample game vector.

In [39]:
embeddings[0]

array([ 0.24690884, -0.60024494, -0.34321675,  0.18282364, -1.2370884 ,
       -0.5774308 , -0.35622823,  0.54888403,  0.28783935, -0.828556  ,
        0.35981768, -0.5272332 , -0.5084524 , -0.7428038 ,  0.12216418,
        0.47244594, -0.52268714, -0.39862898, -1.503811  ,  1.1665981 ,
       -0.24812686, -1.1389667 ,  1.6629837 , -0.37311617,  0.00638717,
       -0.07876787, -0.08371691,  0.39954203, -0.34470356,  0.6996237 ],
      dtype=float32)

To retrieve the name of the game, we first look up the game id using our interactions matrix and then obtain the name using the games dictionary.

In [53]:
firstgameid = interactions.columns[0]
games_dict[firstgameid]

'Counter-Strike'

### Pair similarity

Let us examine how close or distant pairs of games are in the embedding space. The embedding space can be interpreted as a distance metric and so we would expect similar games to have similar vectors.

Our first game will be `Counter-Strike` as seen above. 

We would expect it to be similar to `Left 4 Dead 2`.

In [86]:
# Get data for both games
gamesdata[(gamesdata['title']=='Counter-Strike') | (gamesdata['title']=='Left 4 Dead 2') ]

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
31529,Valve,['Action'],Left 4 Dead 2,Left 4 Dead 2,http://store.steampowered.com/app/550/Left_4_D...,2009-11-16,"['Zombies', 'Co-op', 'FPS', 'Multiplayer', 'Ac...",NaN,http://steamcommunity.com/app/550/reviews/?bro...,"['Single-player', 'Multi-player', 'Co-op', 'St...",19.99,False,550.0,Valve,Overwhelmingly Positive,89.0
32106,Valve,['Action'],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,False,10.0,Valve,Overwhelmingly Positive,88.0


Let us look at the vectors for these two games.

In [70]:
# Set index for Counter-Strike
cs_index = 0

# Obtain embeddings vector 
cs_vector = embeddings[cs_index]

cs_vector

array([ 0.24690884, -0.60024494, -0.34321675,  0.18282364, -1.2370884 ,
       -0.5774308 , -0.35622823,  0.54888403,  0.28783935, -0.828556  ,
        0.35981768, -0.5272332 , -0.5084524 , -0.7428038 ,  0.12216418,
        0.47244594, -0.52268714, -0.39862898, -1.503811  ,  1.1665981 ,
       -0.24812686, -1.1389667 ,  1.6629837 , -0.37311617,  0.00638717,
       -0.07876787, -0.08371691,  0.39954203, -0.34470356,  0.6996237 ],
      dtype=float32)

In [76]:
# Retrieve game id for LFD2
lfd2_id = gamesdata[gamesdata['title']=='Left 4 Dead 2']['id'].values[0]

# Obtain index for Squad in interactions matrix
lfd2_index = list(interactions.columns).index(lfd2_id)

# Obtain embeddings vector
lfd2_vector = embeddings[lfd2_index]

lfd2_vector

array([-0.47054738, -0.09553305,  0.3035873 , -0.04027557, -0.30052912,
       -0.04460974, -0.430263  ,  0.3894665 , -0.16456904, -0.3458131 ,
        0.02933553,  0.08092439, -0.44579884,  0.5828283 ,  0.12577946,
        0.48369217, -0.32890347,  0.08169048,  0.05243007, -0.24915524,
        0.33367637, -0.5070402 ,  0.1763026 , -0.35381997, -0.1324723 ,
        0.40483642,  0.12619357,  0.31072268,  0.07221867,  0.37287298],
      dtype=float32)

To assign a single value to the similarity between these two vectors, we calculate the distance between them. Let us first compute the Euclidean distance.

In [77]:
# Compute Euclidean distance
distance.euclidean(cs_vector, lfd2_vector)

3.6518919467926025

Let us compare this figure with a pair of games we believe to be very different.

In [87]:
# Get data for both games
gamesdata[(gamesdata['title']=='Counter-Strike') | (gamesdata['title']=='The Room') ]

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
2472,Fireproof Games,"['Adventure', 'Indie']",The Room,The Room,http://store.steampowered.com/app/288160/The_R...,2014-07-28,"['Puzzle', 'Adventure', 'Point & Click', 'Indi...",NaN,http://steamcommunity.com/app/288160/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",4.99,False,288160.0,Fireproof Games,Overwhelmingly Positive,73.0
32106,Valve,['Action'],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,False,10.0,Valve,Overwhelmingly Positive,88.0


In [81]:
# Retrieve game id for The Room
room_id = gamesdata[gamesdata['title']=='The Room']['id'].values[0]

# Obtain index for Squad in interactions matrix
room_index = list(interactions.columns).index(room_id)

# Obtain embeddings vector
room_vector = embeddings[room_index]

room_vector

array([ 0.18102524,  0.03450267, -0.49559462,  0.52064276, -0.13829972,
       -0.6893131 , -0.10249795, -0.6738657 , -0.13697192,  0.5107905 ,
       -0.69814545, -0.17198849,  0.00435968,  0.63461906,  0.9719135 ,
        0.17290895,  0.71184224, -0.12161961,  0.43436503, -0.6281764 ,
       -0.11903907, -0.1752168 ,  0.03669906, -0.4266076 , -0.34094498,
        0.25538862,  0.28402406,  0.42189395, -0.7988556 ,  0.30456227],
      dtype=float32)

In [82]:
# Compute Euclidean distance
distance.euclidean(cs_vector, room_vector)

4.734748363494873

When looking at the similarity of embeddings, it is more common to look at cosine similarity. Cosine similarity ranges between -1 and 1 based on the angle between the vectors. From this figure, cosine distance is defined as 1 minus the cosine similarity and therefore a value between 0 and 2.

In [97]:
print(f'Cosine distance between Counter Strike and Left 4 Dead 2: {distance.cosine(cs_vector, lfd2_vector)}')
print(f'Cosine distance between Counter Strike and The Room: {distance.cosine(cs_vector, room_vector)}')

Cosine distance between Counter Strike and Left 4 Dead 2: 0.7091985046863556
Cosine distance between Counter Strike and The Room: 1.108935683965683


### Exploring embeddings with Gensim

In [99]:
# Set threshold to only include games which are owned by at least 200 users

threshold = 200

In [103]:
# Set embedding size
embedding_size = embeddings.shape[1]

In [104]:
# Create instance
kv = WordEmbeddingsKeyedVectors(embedding_size)

In [117]:
# Get list of game names in correct order to match embeddings

gameslist = []
for game_id in interactions.columns:
    name = games_dict[game_id]
    gameslist.append(name)
    
gameslist[:5]

['Counter-Strike',
 'Team Fortress Classic',
 'Day of Defeat',
 'Deathmatch Classic',
 'Half-Life: Opposing Force']

In [118]:
# Add to kv
kv.add(gameslist, embeddings )

In [119]:
kv.most_similar('Counter-Strike')

[('Counter-Strike: Condition Zero', 0.9947258234024048),
 ('Day of Defeat', 0.8795914649963379),
 ('Ricochet', 0.8772425651550293),
 ('Deathmatch Classic', 0.8760509490966797),
 ('Counter-Strike: Source', 0.7674432992935181),
 ('Team Fortress Classic', 0.7315292954444885),
 ('Day of Defeat: Source', 0.7312443852424622),
 ('Half-Life: Source', 0.7133708596229553),
 ('Half-Life: Blue Shift', 0.7080429196357727),
 ('Half-Life: Opposing Force', 0.7068089246749878)]

## User Recommendations

### Recommendations for existing user

In [30]:
# Get recommendations
rec_list_u12 = get_recs(model = mf_model, 
                    interactions = interactions, 
                    user_id = 12, 
                    user_dict = user_dict,
                    item_dict = games_dict, 
                    threshold = 0,
                    num_items = 5,
                    show_known = True, 
                    show_recs = True)

Known Likes:
1- Epic Battle Fantasy 3
2- Paladins®
3- Shakes and Fidget
4- Tactical Craft Online
5- Crusaders of the Lost Idols
6- Time Clickers
7- Dragon Saga
8- Blender
9- Clicker Heroes
10- Spooky's Jump Scare Mansion
11- Codename CURE
12- Fire With Fire Tower Attack and Defense
13- Modular Combat
14- AdVenture Capitalist
15- BrainBread 2
16- Brick-Force
17- Dirty Bomb®
18- Five Nights at Freddy's 2
19- TERA
20- Don't Starve Together
21- Double Action: Boogaloo
22- The Way of Life Free Edition
23- Unturned
24- Trove
25- BLOCKADE 3D
26- Robocraft
27- Block N Load
28- Brawlhalla
29- Warface
30- Quake Live™
31- Lambda Wars Beta
32- Fistful of Frags
33- GunZ 2: The Second Duel
34- BattleBlock Theater®
35- Dungeon Defenders II
36- Warframe
37- Sven Co-op
38- Defiance
39- No More Room in Hell
40- PlanetSide 2
41- Gear Up
42- The Lord of the Rings Online™
43- Blacklight: Retribution
44- Gotham City Impostors Free to Play
45- Everquest ®
46- Castle Crashers®
47- Orcs Must Die! 2
48- Realm o

In [ ]:
### Add comments - Expect additional CS game etc

### Recommendations for new user

In [8]:
# Get list of owned games from user

# Create empty list to store game ids
ownedgames = []

# Create loop to prompt for game id and ask if want to continue
moretoadd = 'y'
while moretoadd == 'y':
    game = input('Please enter the game id: ')
    ownedgames.append(game)
    moretoadd = input('Do you have more games to add? y/n ')
    
# Print list of owned games
print(f"You own the following games: {ownedgames}")

Please enter the game id: 8
Do you have more games to add? y/n n
You own the following games: ['8']


In [15]:
# View last entry in interactions matrix
interactions[-1:]

id,10,20,30,40,50,60,70,80,130,220,...,526790,527340,527440,527510,527520,527810,527890,527900,528660,530720
uid,,,,,,,,,,,,,,,,,,,,,
88308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
 # Get new user index
newuidindex = interactions.index[-1]+1
newuidindex

88309

In [17]:
# Add row for new user id with zeros
interactions.loc[newuidindex] = 0
interactions

id,10,20,30,40,50,60,70,80,130,220,...,526790,527340,527440,527510,527520,527810,527890,527900,528660,530720
uid,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# Change value to 1 where user owns the game

for gameid in ownedgames:
    interactions.loc[newuidindex, gameid] = 1

In [ ]:
# Recreate dictionaries
user_dict = create_user_dict(interactions=interactions)
games_dict = create_item_dict(df = games, id_col = 'id', name_col = 'title')

In [ ]:
# Fit model
mf_model = run_model(interactions = train,
                     n_components = 30,
                     loss = 'warp',
                     epoch = 30,
                     n_jobs = 4)

In [ ]:
# Get recommendations
rec_list = get_recs(model = mf_model, 
                    interactions = interactions, 
                    user_id = newuidindex, 
                    user_dict = user_dict,
                    item_dict = games_dict, 
                    threshold = 0,
                    num_items = 5,
                    show_known = True, 
                    show_recs = True)

## Item Recommendations

### Create item embedding matrix

In [64]:
item_dist = create_item_emdedding_matrix(model = mf_model,interactions = interactions)

In [66]:
item_dist.shape

(8791, 8791)

In [67]:
item_dist.head()

id,10,20,30,40,50,60,70,80,130,220,...,526790,527340,527440,527510,527520,527810,527890,527900,528660,530720
id,,,,,,,,,,,,,,,,,,,,,
10,1.000000,0.703092,0.867419,0.871559,0.684434,0.869805,0.683555,0.993782,0.683072,0.510813,...,0.041585,0.070690,-0.076905,-0.086024,0.070537,0.008223,-0.148230,-0.115156,0.014385,-0.056317
20,0.703092,1.000000,0.845572,0.843557,0.997379,0.839401,0.986706,0.663473,0.997265,0.847267,...,-0.166892,-0.104024,-0.078415,-0.054945,-0.042555,-0.072952,-0.243327,-0.226339,-0.004695,-0.075152
30,0.867419,0.845572,1.000000,0.996962,0.839656,0.996300,0.812246,0.819267,0.840940,0.617789,...,0.057061,-0.125987,0.019689,-0.035818,0.096059,0.076818,-0.079370,-0.075650,0.087320,0.050368
40,0.871559,0.843557,0.996962,1.000000,0.838675,0.999104,0.810467,0.822601,0.839866,0.608376,...,0.067647,-0.125478,0.008246,-0.021051,0.112209,0.093214,-0.075856,-0.070456,0.091169,0.053585
50,0.684434,0.997379,0.839656,0.838675,1.000000,0.834340,0.989329,0.641949,0.999489,0.859258,...,-0.184936,-0.122161,-0.089101,-0.062516,-0.046453,-0.075597,-0.248418,-0.238196,-0.020810,-0.079319


### Generate item recommendations

In [70]:
# Similar items to item_id 10 - Counter Strike
item_rec_list_10 = get_item_recs(item_emdedding_matrix = item_dist, 
                              item_id = 10, 
                              item_dict = games_dict, 
                              n_items = 10, 
                              show = True)

Item of interest: Counter-Strike
Similar items:
1- Counter-Strike: Condition Zero
2- Deathmatch Classic
3- Ricochet
4- Day of Defeat
5- Counter-Strike: Source
6- Team Fortress Classic
7- Day of Defeat: Source
8- Half-Life: Opposing Force
9- Half-Life
10- Half-Life: Blue Shift


In [71]:
games[games['title'] == 'The Witness']

,title,id
5211,The Witness,210970.0


In [72]:
# Similar items to item_id 210970 The Witness
item_rec_list = get_item_recs(item_emdedding_matrix = item_dist, 
                              item_id = 210970, 
                              item_dict = games_dict, 
                              n_items = 10, 
                              show = True)

Item of interest: The Witness
Similar items:
1- Headlander
2- ABZU
3- SUPERHOT
4- Assault Android Cactus
5- Oxenfree
6- Quadrilateral Cowboy
7- Poly Bridge
8- The Guest
9- Firewatch
10- Grow Up


In [78]:
gamesdata = pd.read_csv('gamesdata.csv', index_col = 0)
gamesdata[gamesdata['id'].isin(item_rec_list)]

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
1550,Witch Beam,"['Action', 'Indie']",Assault Android Cactus,Assault Android Cactus,http://store.steampowered.com/app/250110/Assau...,2015-09-23,"['Twin Stick Shooter', 'Indie', 'Action', 'Bul...",NaN,http://steamcommunity.com/app/250110/reviews/?...,"['Single-player', 'Shared/Split Screen', 'Stea...",14.99,False,250110.0,Witch Beam,Overwhelmingly Positive,79.0
5143,Night School Studio,"['Adventure', 'Indie']",Oxenfree,Oxenfree,http://store.steampowered.com/app/388880/Oxenf...,2016-01-14,"['Story Rich', 'Choices Matter', 'Supernatural...",NaN,http://steamcommunity.com/app/388880/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",19.99,False,388880.0,Night School Studio,Very Positive,80.0
5500,SUPERHOT Team,"['Action', 'Indie']",SUPERHOT,SUPERHOT,http://store.steampowered.com/app/322500/SUPER...,2016-02-25,"['FPS', 'Action', 'Bullet Time', 'First-Person...",NaN,http://steamcommunity.com/app/322500/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",24.99,False,322500.0,SUPERHOT Team,Very Positive,82.0
5513,505 Games,"['Adventure', 'Indie']",The Guest,The Guest,http://store.steampowered.com/app/402040/The_G...,2016-03-10,"['Indie', 'Adventure', 'Puzzle', 'Atmospheric'...",NaN,http://steamcommunity.com/app/402040/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",9.99,False,402040.0,Team Gotham,Mostly Positive,68.0
6783,Adult Swim Games,['Action'],Headlander,Headlander,http://store.steampowered.com/app/340000/Headl...,2016-07-25,"['Action', 'Sci-fi', 'Metroidvania', 'Platform...",NaN,http://steamcommunity.com/app/340000/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",19.99,False,340000.0,Double Fine Productions,Very Positive,74.0
6976,Ubisoft,"['Adventure', 'Casual', 'Indie']",Grow Up,Grow Up,http://store.steampowered.com/app/426790/Grow_Up/,2016-08-16,"['Adventure', 'Casual', 'Indie', 'Open World',...",NaN,http://steamcommunity.com/app/426790/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",9.99,False,426790.0,"Reflections, a Ubisoft Studio",Very Positive,71.0
22123,505 Games,"['Action', 'Adventure', 'Casual', 'Indie', 'Si...",ABZU,ABZU,http://store.steampowered.com/app/384190/ABZU/,2016-08-02,"['Relaxing', 'Atmospheric', 'Underwater', 'Gre...",NaN,http://steamcommunity.com/app/384190/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",19.99,False,384190.0,Giant Squid,Very Positive,83.0
22244,Blendo Games,"['Action', 'Adventure', 'Indie']",Quadrilateral Cowboy,Quadrilateral Cowboy,http://store.steampowered.com/app/240440/Quadr...,2016-07-25,"['Indie', 'Adventure', 'Action', 'Hacking', 'C...",NaN,http://steamcommunity.com/app/240440/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",19.99,False,240440.0,Blendo Games,Very Positive,81.0
22364,Dry Cactus,"['Indie', 'Simulation']",Poly Bridge,Poly Bridge,http://store.steampowered.com/app/367450/Poly_...,2016-07-12,"['Building', 'Simulation', 'Physics', 'Puzzle'...",NaN,http://steamcommunity.com/app/367450/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",11.99,False,367450.0,Dry Cactus,Very Positive,73.0
24216,"Panic Inc., Campo Santo","['Adventure', 'Indie']",Firewatch,Firewatch,http://store.steampowered.com/app/383870/Firew...,2016-02-09,"['Atmospheric', 'Adventure', 'Walking Simulato...",NaN,http://steamcommunity.com/app/383870/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",19.99,False,383870.0,Campo Santo,Very Positive,81.0
